In [3]:
import numpy as np

import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.applications.xception import preprocess_input

2022-11-22 15:22:28.647447: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-22 15:22:28.647485: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [4]:
import tensorflow.lite as tflite

interpreter = tflite.Interpreter(model_path='duck-model.tflite')
interpreter.allocate_tensors()

input_index = interpreter.get_input_details()[0]['index']
output_index = interpreter.get_output_details()[0]['index']

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [5]:
interpreter.get_input_details()

[{'name': 'serving_default_input_23:0',
  'index': 0,
  'shape': array([  1, 260, 260,   3], dtype=int32),
  'shape_signature': array([ -1, 260, 260,   3], dtype=int32),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]

In [6]:
interpreter.get_output_details()

[{'name': 'StatefulPartitionedCall:0',
  'index': 694,
  'shape': array([1, 3], dtype=int32),
  'shape_signature': array([-1,  3], dtype=int32),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]

In [7]:
class_names = ['Northern_mockingbird', 'Red_headed_Woodpecker', 'Wood_duck']

def interpret_image(image_file):
    ### Load an Image Red_headed_woodpecker
    img = load_img(image_file, target_size=(260, 260))

    ### Remind that rescaling input is skipped for EfficientNet
    x = np.array(img, dtype='float32')
    X = np.array([x])
    
    interpreter.set_tensor(input_index, X)
    interpreter.invoke()
    pred_result =  interpreter.get_tensor(output_index)
    
    return dict(zip(class_names, pred_result[0]))

In [8]:
interpret_image('YRKTWVRKWHY5.jpg')

{'Northern_mockingbird': 0.2838146,
 'Red_headed_Woodpecker': 0.59998286,
 'Wood_duck': 0.11620256}

In [9]:
interpret_image('W056SSPR5L7L.jpg')

{'Northern_mockingbird': 0.73422086,
 'Red_headed_Woodpecker': 0.12016131,
 'Wood_duck': 0.1456178}

In [10]:
interpret_image('Y6SS2NGCG0NS.jpg')

{'Northern_mockingbird': 0.19016749,
 'Red_headed_Woodpecker': 0.16592956,
 'Wood_duck': 0.64390296}